In [1]:
# Here, I'm going to get a list of DEF CON Youtube videos, and use Linear Optimization to select videos for myself
## install google python API: https://developers.google.com/api-client-library/python/

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd

In [2]:
# steps to get API Key: https://developers.google.com/youtube/v3/getting-started

API_KEY = "[YOUR API KEY]"   # use your own API key
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

In [3]:
# Get videos, channel, playlists that match your search term
def youtube_search(search_query, max_results):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
        developerKey=API_KEY)

    # Call the search.list method to search specified query
    search_response = youtube.search().list(
        q=search_query,
        part='id,snippet',
        maxResults=max_results
      ).execute()

    videos = []
    channels = []
    playlists = []

    # You can print out search_response here and read it through JSON viewer: http://jsonviewer.stack.hu/
    for search_result in search_response.get('items', []):
        if search_result['id']['kind'] == 'youtube#video':
            videos.append('%s (%s)' % (search_result['snippet']['title'],
                                         search_result['id']['videoId']))
        elif search_result['id']['kind'] == 'youtube#channel':
            channels.append('%s (%s)' % (search_result['snippet']['title'],
                                           search_result['id']['channelId']))
        elif search_result['id']['kind'] == 'youtube#playlist':
            playlists.append('%s (%s)' % (search_result['snippet']['title'],
                                            search_result['id']['playlistId']))
    return videos, channels, playlists

In [4]:
search_query = 'DEF CON 25 SE'
max_results = 50  # at most 50, limited by Google

vlst, clst, plst = youtube_search(search_query, max_results)

In [5]:
print len(vlst), len(clst), len(plst)

47 1 2


In [6]:
for elem in vlst:
    print elem
print
    
for elem in clst:
    print elem
print
    
for elem in plst:
    print elem

DEF CON 25 SE Village - Jayson Street - Strategies on Securing Your Banks and Enterprises (iLPI0EGs6kY)
DEF CON 25 SE Village - Keith Conway, Cameron Craig - Change Agents  How to Affect  Corp. Culture (mYRXlCGMP0Q)
DEF CON 25 - Max Bazaliy - Jailbreaking Apple Watch (eJpbi-Qz6Jc)
DEF CON 25 - Nathan Seidle - Open Source Safe Cracking Robots (v9vIcfLrmiA)
DEF CON 24 - Marc Newlin - MouseJack: Injecting Keystrokes into Wireless Mice (00A36VABIA4)
DEF CON 25 SE Village - Brent White, Tim Roberts - Skills For A Red Teamer (yJCq8ajdd0U)
DEF CON 25 SE Village - Michele Fincher - Are You Killing Your Security Program? (AZw8WugNyIQ)
DEF CON 25 SE Village - Chris Hadnagy - SE vs Predator: Using SE In Ways I Never Thought (PNuPkpLuaws)
DEF CON 25 SE Village - Robert Wood  - Thematic Social Engineering (Rw0aJQnYXq0)
DEF CON 25 SE Village - John Nye -The Human Factor  Why Are We So Bad at Security (MgXhjUzi_I0)
DEF CON 24 SE Village - Chris Hadnagy - 7 Jedi Mind Tricks: Influence Your Target with

In [7]:
def videos_list_by_id(**kwargs):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
        developerKey=API_KEY)
    result = youtube.videos().list(**kwargs).execute().get('items')[0]

    comment_count = result['statistics']['commentCount']
    view_count = result['statistics']['viewCount']
    favorite_count = result['statistics']['favoriteCount']
    like_count = result['statistics']['likeCount']
    description = result['snippet']['localized']['description']
    duration = result['contentDetails']['duration']
    
    return {'id':kwargs['id'], 'comment_count':comment_count, 'view_count':view_count, 'favorite_count':favorite_count,
           'like_count':like_count, 'duration':duration, 'description':description}

In [8]:
video_dct = {}

idx = 0
for elem in vlst:
    title, vid = elem.split('(')
    vid = vid.split(')')[0]
    item_dct = videos_list_by_id(part='snippet,contentDetails,statistics', id=vid)
    item_dct['title'] = title
    video_dct[idx] = item_dct
    idx += 1

In [9]:
df = pd.DataFrame(video_dct.values())
df.head()

,comment_count,description,duration,favorite_count,id,like_count,title,view_count
0,10,Most people who work on the defensive side of ...,PT56M20S,0,iLPI0EGs6kY,62,DEF CON 25 SE Village - Jayson Street - Strate...,1358
1,2,It’s no secret that trying to change corporate...,PT46M13S,0,mYRXlCGMP0Q,32,"DEF CON 25 SE Village - Keith Conway, Cameron ...",1195
2,22,"On April 24, 2015, Apple launched themselves i...",PT36M12S,0,eJpbi-Qz6Jc,287,DEF CON 25 - Max Bazaliy - Jailbreaking Apple ...,28025
3,57,We've built a $200 open source robot that crac...,PT27M25S,0,v9vIcfLrmiA,826,DEF CON 25 - Nathan Seidle - Open Source Safe ...,31403
4,47,What if your wireless mouse was an effective a...,PT42M35S,0,00A36VABIA4,479,DEF CON 24 - Marc Newlin - MouseJack: Injectin...,41320


In [9]:
from pulp import *